In [12]:
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn import datasets
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.svm import LinearSVR
from sklearn.svm import SVR

# load iris data
iris = datasets.load_iris()
X = iris["data"][:,(2,3)] # petal length, petal width
y = (iris["target"] == 2).astype(np.float64) # Iris-Virginica

# linear SVM
svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("linear_svc", LinearSVC(C=1, loss="hinge")),
])
svm_clf.fit(X, y)
X_1 = np.linspace(0, 7, 200).reshape(-1,1)
X_2 = np.linspace(0, 3.5, 200).reshape(-1,1)
X_1_new, X_2_new = np.meshgrid(np.arange(0, 7, 0.1), np.arange(0, 3.5, 0.1))
X1X2 = np.array([X_1_new.flatten(),X_2_new.flatten()]).T
y_predict = svm_clf.predict(X1X2)
plt.figure()
plt.contourf(X_1_new, X_2_new, y_predict.reshape(X_1_new.shape), 
             cmap=ListedColormap(['#FF0000', '#0000FF']), alpha=0.5)
Xa = X[y==1]
Xb = X[y==0]
plt.plot(Xa[:,0],Xa[:,1], "b.", marker="s")
plt.plot(Xb[:,0],Xb[:,1], "r.", marker="o")
plt.xlabel("Petal length (cm)")
plt.ylabel("Petal width (cm)")
plt.title("Linear SVC")
plt.savefig("../plots/ex_5_01.pdf")

# non-linear SVM
X, y = make_moons(noise=0.3, random_state=0)
polynomial_svm_clf = Pipeline([
    ("poly_feature", PolynomialFeatures(degree=3)),
    ("scaler", StandardScaler()),
    ("svm_clf", LinearSVC(C=10, loss="hinge"))
])
polynomial_svm_clf.fit(X,y)
X1, X2 = np.meshgrid(np.arange(-1.5,2.5,0.05), np.arange(-1.5,2.0,0.05))
X1X2 = np.array([X1.flatten(), X2.flatten()]).T
y_predict = polynomial_svm_clf.predict(X1X2)
plt.figure()
plt.contourf(X1, X2, y_predict.reshape(X1.shape), 
            cmap=ListedColormap(['#FF0000', '#0000FF']), alpha=0.5)
Xa = X[y==1]
Xb = X[y==0]
plt.plot(Xa[:,0],Xa[:,1], "r.", marker="s")
plt.plot(Xb[:,0],Xb[:,1], "b.", marker="o")
plt.xlabel("X_1")
plt.ylabel("X_2")
plt.title("Linear SVC with polynomial")
plt.savefig("../plots/ex_5_02.pdf")

# Polynomial Kernel
poly_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="poly", degree=5, coef0=10, C=5)),
])
poly_kernel_svm_clf.fit(X,y)
y_predict = poly_kernel_svm_clf.predict(X1X2)
plt.figure()
plt.contourf(X1, X2, y_predict.reshape(X1.shape), 
            cmap=ListedColormap(['#FF0000', '#0000FF']), alpha=0.5)
plt.plot(Xa[:,0],Xa[:,1], "r.", marker="s")
plt.plot(Xb[:,0],Xb[:,1], "b.", marker="o")
plt.xlabel("X_1")
plt.ylabel("X_2")
plt.title("Polynomial Kernel")
plt.savefig("../plots/ex_5_03.pdf")

# Gaussian Radial Basis Function (RBF) Kernel
rbf_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="rbf", gamma=5, C=0.001)),
])
rbf_kernel_svm_clf.fit(X,y)
y_predict = rbf_kernel_svm_clf.predict(X1X2)
plt.figure()
plt.contourf(X1, X2, y_predict.reshape(X1.shape), 
            cmap=ListedColormap(['#FF0000', '#0000FF']), alpha=0.5)
Xa = X[y==1]
Xb = X[y==0]
plt.plot(Xa[:,0],Xa[:,1], "r.", marker="s")
plt.plot(Xb[:,0],Xb[:,1], "b.", marker="o")
plt.xlabel("X_1")
plt.ylabel("X_2")
plt.title("RBF kernel")
plt.savefig("../plots/ex_5_04.pdf")

# Linear SVM Regression
X = 2 * np.random.rand(100,1)
y = 4 + 3 * X + 5 * np.random.rand(100,1)
svm_reg = LinearSVR(epsilon=1.5)
svm_reg.fit(X, y)
y_predict_e1 = svm_reg.predict(np.arange(0,2,0.05).reshape(-1,1))
svm_reg = LinearSVR(epsilon=0.5)
svm_reg.fit(X, y)
y_predict_e2 = svm_reg.predict(np.arange(0,2,0.05).reshape(-1,1))
plt.figure()
plt.plot(X, y, "b.", label="data")
plt.plot(np.arange(0,2,0.05), y_predict_e1, "r-", linewidth=2, label="epsilon = 1.5")
plt.plot(np.arange(0,2,0.05), y_predict_e2, "g-", linewidth=2, label="epsilon = 0.5")
plt.xlabel("X")
plt.ylabel("y")
plt.legend()
plt.savefig("../plots/ex_5_05.pdf")

# Nonlinear SVM Regression
m = 100
X = 6 * np.random.rand(m, 1) - 3
y = 0.5 * X**2 + X + 2 + np.random.randn(m, 1)
svm_poly_reg = SVR(kernel="poly", degree=2, C=100, epsilon=0.1)
svm_poly_reg.fit(X,y)
y_predict_e1 = svm_poly_reg.predict(np.arange(-3,3,0.05).reshape(-1,1))
svm_poly_reg = SVR(kernel="poly", degree=2, C=0.01, epsilon=0.1)
svm_poly_reg.fit(X,y)
y_predict_e2 = svm_poly_reg.predict(np.arange(-3,3,0.05).reshape(-1,1))
plt.figure()
plt.plot(X, y, "b.", label="data")
plt.plot(np.arange(-3,3,0.05), y_predict_e1, "r-", linewidth=2, label="C = 100")
plt.plot(np.arange(-3,3,0.05), y_predict_e2, "g-", linewidth=2, label="C = 0.01")
plt.xlabel("X")
plt.ylabel("y")
plt.legend()
plt.savefig("../plots/ex_5_06.pdf")
